In [ ]:
# imports
import numpy as np
import gc
import matplotlib.pyplot as plt
from sklearn_sfa_master.sksfa import HSFA
import pickle

In [ ]:
# Load data & crop / reduce size if dim_step > 1
data = np.load('data/data_squareRoom.npy')
crop_h = 20
crop_w = 10
dim_step= 1
data = data[:, crop_h:-crop_h, crop_w:-crop_w][:, ::dim_step][:, :, ::dim_step]
gc.collect()
print(data.shape)

In [ ]:
# standardize images
images = np.array([(image - image.min()) / (image.max() - image.min()) for image in data])
print(images[0].shape)

In [ ]:
# init. HSFA (Skip this step if you want to load the pre-trained model)
layer_configurations = [(10, 10, 5, 5, 5, 2)]
hsfa = HSFA(10, images.shape[1:], layer_configurations, verbose=True)
hsfa = hsfa.fit(images)
slow_features = hsfa.transform(images)

with open("hsfa.pkl", "wb") as file:
    pickle.dump(hsfa, file)
np.save("slow_features.npy", slow_features)

In [ ]:
# Load pre-trained hsfa
with open("hsfa.pkl", "rb") as file:
    hsfa = pickle.load(file)
print(hsfa.summary())
slow_features = np.load("slow_features.npy")

In [ ]:
# Plot 5 slowest features
fig = plt.figure(figsize=(20, 6))
ax = fig.add_subplot()

# Compare two images 
index1 = 67
index2 = index1+1

ax.plot(slow_features[:, 0], label="Slow Feature 1")
ax.plot(slow_features[:, 1], label="Slow Feature 2")
ax.plot(slow_features[:, 2], label="Slow Feature 3")
ax.plot(slow_features[:, 3], label="Slow Feature 4")
print('pos 1:', slow_features[index1])
print('pos 2:', slow_features[index2])
print('Change:', np.abs(slow_features[index1] - slow_features[index2]))
print(f'Max diff.: Slow feature #{np.argmax(np.abs(slow_features[index1] - slow_features[index2])) + 1}')
ax.legend()
ax.grid(alpha=.3)
hline = ax.axvline(index1, c='r', alpha=.5, linestyle='--')
hline = ax.axvline(index2, c='g', alpha=.5, linestyle='--')
plt.title("Extracted Slow Features")
plt.xlabel("Frame")
plt.ylabel("Feature Value")
length = 200
steps = 4
plt.xlim(0,length)
plt.xticks(np.arange(0, length, steps))

plt.figure(figsize=(10, 5))
plt.imshow(data[index1])
plt.figure(figsize=(10, 5))
plt.imshow(data[index2])
plt.show()

In [ ]:
# Scatter plot of first 4 features (not really useful)
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
ax.scatter(slow_features[:,0], slow_features[:,1], slow_features[:,2], c=slow_features[:,3], cmap='viridis', alpha=0.6)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# init. SOM & find winners

from minisom import MiniSom
from tqdm.notebook import tqdm
print(round(np.sqrt(100**2 + 100**2)))
som = MiniSom(200, 200, 9, sigma=3, learning_rate=0.3, random_seed=42)
som.random_weights_init(slow_features[:,1:10])
som.train(slow_features[:,1:10], 5000, random_order=True, verbose=True)
winners = [som.winner(x) for x in tqdm(slow_features[:,1:10])]

In [ ]:
# plot SOM & compare two images

index = 45
index2 = index+1
winners = np.array(winners)

# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(projection='3d')
# ax.scatter(winners[:,0], winners[:,1], slow_features[:,0], marker='.', c='black', alpha=.1)
# ax.plot(winners[index,0], winners[index,1], slow_features[index,0], 'ro', label='first image')
# ax.plot(winners[index2,0], winners[index2,1], slow_features[index2,0], 'bo', label='second image')
# ax.legend()
# ax.view_init(elev=30, azim=-45)
# plt.xlabel('x')
# plt.ylabel('y')


plt.figure(figsize=(10, 10))
combined = slow_features[:,0]
scaledRotation = (combined - np.min(combined)) / (np.max(combined) - np.min(combined))

plt.scatter(winners[:,0], winners[:,1], marker='.', c=scaledRotation, cmap='viridis', alpha=.6)
cbar = plt.colorbar()
plt.plot(winners[index,0], winners[index,1], 'ro', label='first image', markersize=10)
plt.plot(winners[index2,0], winners[index2,1], c='orange', marker='o', label='second image', markersize=10)
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.title("SOM w/ 4 features (rotation = color)")
plt.figure(figsize=(10, 6))
plt.imshow(data[index])
plt.xticks([])
plt.yticks([])
plt.figure(figsize=(10, 6))
plt.imshow(data[index2])
plt.xticks([])
plt.yticks([])
plt.show()

<h1>Use own recorded data</h1>

In [ ]:
# Load recorded data & stitch them together, outcome: dataset w/ shape (3000, 30, 190, 3)
# & perform SFA

recorded_data = []
pos_data = []

camera_only_rot = np.load('recorded_data/dataset/camera_only_rot.npy')
camera_no_rot = np.load('recorded_data/dataset/camera_no_rot.npy')
camera = np.load('recorded_data/dataset/camera.npy')

pos_only_rot = np.load('recorded_data/dataset/position_only_rot.npy')
pos_no_rot = np.load('recorded_data/dataset/position_no_rot.npy')
pos = np.load('recorded_data/dataset/position.npy')

dataset = np.zeros((3000, 30, 190, 3))
positions = np.zeros((3000,4))

for i in range(len(dataset)):
    if i < 1000:
        dataset[i] = camera_only_rot[i]
        positions[i] = pos_only_rot[i]
    elif i < 2000:
        dataset[i] = camera_no_rot[i-1000]
        positions[i] = pos_no_rot[i-1000]
    else:
        dataset[i] = camera[i-2000]
        positions[i] = pos[i-2000]

images_own = np.array([(image - image.min()) / (image.max() - image.min()) for image in dataset])
print(images_own.shape)
slow_features_own_dataset = hsfa.transform(images_own)


In [ ]:
# Plot three slowest features
plt.figure(figsize=(20, 6))
plt.plot(slow_features_own_dataset[:,0], label='sf#1')
plt.plot(slow_features_own_dataset[:,1])
plt.plot(slow_features_own_dataset[:,2])
plt.xlim([0, 200])
plt.grid()
plt.legend()
plt.figure()
plt.imshow(images[25])
plt.xticks([])
plt.yticks([])
plt.figure()
plt.imshow(images[26])
plt.xticks([])
plt.yticks([])
plt.show()
print(positions[100], positions[101])

In [ ]:
# SOM for recorded data
winners = [som.winner(x) for x in tqdm(slow_features_own_dataset[:,1:10])]
winners = np.array(winners)
plt.figure(figsize=(10, 10))
plt.scatter(winners[:,0], winners[:,1], c=slow_features_own_dataset[:,0], cmap='viridis', marker='.', alpha=1)
plt.title("Pre-trained pipeline (HSFA [10 features], SOM) applied to own dataset")
plt.plot()
plt.colorbar()

index = 25
index2 = 26

plt.plot(winners[index,0], winners[index,1], 'ro', label='first image', markersize=10)
plt.plot(winners[index2,0], winners[index2,1], c='orange', marker='o', label='second image', markersize=10)
plt.show()